In [1]:
import pandas as pd
import numpy as np
import sqlite3
from tqdm.notebook import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Taxonomy

In [2]:
tax_detail = pd.read_csv('../data/nucc_taxonomy_220.csv') # equal to specialty codes. Each specialty has a specific code
tax_detail.head(5)

,Code,Grouping,Classification,Specialization,Definition,Notes,Display Name,Section
0,193200000X,Group,Multi-Specialty,NaN,A business group of one or more individual pra...,[7/1/2003: new],Multi-Specialty Group,Individual
1,193400000X,Group,Single Specialty,NaN,A business group of one or more individual pra...,[7/1/2003: new],Single Specialty Group,Individual
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,NaN,An allergist-immunologist is trained in evalua...,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician,Individual
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,Definition to come...,NaN,Allergy Physician,Individual
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,Definition to come...,NaN,Clinical & Laboratory Immunology (Allergy & Im...,Individual


# NPI data needed info
- 'NPI'
- 'Entity Type Code' # 1 for provider, 2 for facility
- 'Provider Organization Name (Legal Business Name)'
- 'Provider Last Name (Legal Name)'
- 'Provider First Name'
- 'Provider Middle Name'
- 'Provider Name Prefix Text'
- 'Provider Name Suffix Text'
- 'Provider Credential Text'
- 'Provider First Line Business Practice Location Address'
- 'Provider Second Line Business Practice Location Address'
- 'Provider Business Practice Location Address City Name'
- 'Provider Business Practice Location Address State Name'
- 'Provider Business Practice Location Address Postal Code'
- The provider's taxonomy code

In [3]:
npi = pd.read_csv('../data/npidata_pfile_20050523-20220213.csv', chunksize = 100000)
npi_chunk1 = next(npi)
npi_chunk1.head(1)

/Users/ntaherkh/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (17,24,32,34,72,76,77,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,147,155,159,163,167,169,171,173,174,175,177,178,179,181,182,183,185,186,187,189,190,191,193,194,195,197,198,199,201,202,203,205,206,207,209,210,211,213,214,215,217,218,219,221,222,223,225,226,227,229,230,231,233,234,237,239,241,243,245,247,249,253,257,261,265,269,271,273,277,279,281,285,289,293,319,320,321,322,323,324,325,326,327,328) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Business Mailing Address Telephone Number,Provider Business Mailing Address Fax Number,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Country Code (If outside U.S.),Provider Business Practice Location Address Telephone Number,Provider Business Practice Location Address Fax Number,Provider Enumeration Date,Last Update Date,NPI Deactivation Reason Code,NPI Deactivation Date,NPI Reactivation Date,Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Healthcare Provider Taxonomy Code_1,Provider License Number_1,Provider License Number State Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Provider License Number_2,Provider License Number State Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Provider License Number_3,Provider License Number State Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Provider License Number_4,Provider License Number State Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Provider License Number_5,Provider License Number State Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Provider License Number_6,Provider License Number State Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Provider License Number_7,Provider License Number State Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Provider License Number_8,Provider License Number State Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Provider License Number_9,Provider License Number State Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Provider License Number_10,Provider License Number State Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Provider License Number_11,Provider License Number State Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Provider License Number_12,Provider License Number State Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Provider License Number_13,Provider License Number State Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Provider License Number_14,Provider License Number State Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Provider License Number_15,Provider License Number State Code_15,Healthcare Provider Primary Taxonomy Switch_15,Other Provider Identifier_1,Other Provider Identifier Type Code_1,Other Provider Ide

In [4]:
# part of NPI df that contains the taxonomy codes
taxes = npi_chunk1.filter(regex = '^NPI$|(Taxonomy (Code|Switch))') #only NPI or Taxonomy code or Taxonomy switch
taxes.head(1)

,NPI,Healthcare Provider Taxonomy Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15
0,1679576722,207X00000X,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:

taxes_code_long = taxes.melt(id_vars = [col for col in taxes.columns if 'NPI' in col or 'Taxonomy Switch' in col],
           var_name = 'Taxonomy Code Number',
           value_name = 'Taxonomy Code' #melt is a type of pivoting with null values. 
                             #id_vars is the columns you want on the left. 
                             # the rest of the columns take part in the melt
                            #var_name and value_name are two clumns renamed in the new df with your choice
                             #first one is filled with the column name with the non-null value (check?)
                             # second one is filled with the value for that colum
                            
          )


In [6]:
taxes_code_long.head(1)

,NPI,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Primary Taxonomy Switch_15,Taxonomy Code Number,Taxonomy Code
0,1679576722,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare Provider Taxonomy Code_1,207X00000X


In [28]:

taxes_long = taxes_code_long.melt(id_vars = [col for col in taxes_code_long.columns if 'NPI' in col or 'Taxonomy Code' in col],
                 var_name = 'Taxonomy Switch Number',
                 value_name = 'Taxonomy Switch'
            )
taxes_long.head(1)

,NPI,Taxonomy Code Number,Taxonomy Code,Taxonomy Switch Number,Taxonomy Switch
0,1235896861,Healthcare Provider Taxonomy Code_1,3747P1801X,Healthcare Provider Primary Taxonomy Switch_1,Y
1,1053078683,Healthcare Provider Taxonomy Code_1,363LA2100X,Healthcare Provider Primary Taxonomy Switch_1,Y


At this point, I've melted my data twice making it less wide but much longer. I have a column with a bunch of values that say something like 'Healthcare Provider Taxonomy Code_5' or 'Healthcare Provider Primary Taxonomy Switch_11'. I don't really need all the words, though, since my column labels are already telling me that these are codes or switches; I just need the numbers at the end. I tested a few ways of just getting the numbers, and the most efficient in terms of my computer getting the job done quickly was just grabbing whatever text comes after all the words. So the len('Healthcare Provider Primary Taxonomy Switch_') could just as well have been the number 44, representing the number of characters there. so that could say x[44:] to say "start after all those letters and get me everything to the end"--that is, get me the numbers. The apply( lambda x) basically makes that a function that does the same thing all the way down the dataset.

In [8]:
taxes_long['Taxonomy Code Number'] = taxes_long['Taxonomy Code Number'].apply(lambda x: x[len('Healthcare Provider Taxonomy Code_'):])
taxes_long['Taxonomy Switch Number'] = taxes_long['Taxonomy Switch Number'].apply(lambda x: x[len('Healthcare Provider Primary Taxonomy Switch_'):])

After doing the steps above, I end up with lines where the code number is 5 and the switch number is 9 (or whatever). But the point of those codes and switches is that switch 5 tells you something about code 5, and switch 6 tells you something about code 6, etc. So to make the data make sense, I filter down only to rows where the code and switch match.

In [9]:
primary_taxes =taxes_long[(taxes_long['Taxonomy Code Number'] == taxes_long['Taxonomy Switch Number']) & (taxes_long['Taxonomy Switch'] == 'Y')].drop(columns = ['Taxonomy Code Number','Taxonomy Switch Number'])
            

In [10]:
primary_taxes.head(2)

,NPI,Taxonomy Code,Taxonomy Switch
0,1679576722,207X00000X,Y
1,1588667638,207RC0000X,Y


In [11]:
prof_cols = ['NPI',
                 'Entity Type Code',
                 'Provider Organization Name (Legal Business Name)',
                 'Provider Last Name (Legal Name)',
                 'Provider First Name',
                 'Provider Middle Name',
                 'Provider Name Prefix Text',
                 'Provider Name Suffix Text',
                 'Provider Credential Text',
                 'Provider First Line Business Practice Location Address',
                 'Provider Second Line Business Practice Location Address',
                 'Provider Business Practice Location Address City Name',
                 'Provider Business Practice Location Address State Name',
                 'Provider Business Practice Location Address Postal Code']
prof = npi_chunk1.filter(items = prof_cols)

In [12]:
prof.head(1)

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code
0,1679576722,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,3500 CENTRAL AVE,NaN,KEARNEY,NE,688472944.0


Technically, all zip codes in the US have the form 12345-6789, that is, five digits and then another four digits. Of course, we usually don't use the last four. The formatting of the postal codes in the NPPES data isn't consistent; some use 9 digits and some use just 5. The 9 is there for the cases where they intended to have 9 digits.
Sometimes when we read in zip codes, Python think that those zip codes are numbers and therefore cuts off leading zeros. So a zip code like 06519 might get read in as 6519 and a zip code like 06519-0240 might get read in as 065190240 might get read in as 65190240. In the  6519 case, I'd want to fill in to 5 digits; in the 65190240 case, I'd want to fill in to 9 digits. Once I fill in to 9 digits, I'm only interested in the first five digits, which is what the [:5] is getting me.

In [13]:
def zip_filler(x):
    try:
        if len(x) > 5: #018783840 for example may be read as 18783840
            return x.zfill(9)[:5]
        elif len(x) > 0: # 01878 for example
            return x.zfill(5)
    except:
        return np.nan

In [14]:
prof['Provider Business Practice Location Address Postal Code'] = (prof['Provider Business Practice Location Address Postal Code']
                                                                      .astype(str)
                                                                      .str.replace('.0', '', regex = True)
                                                                      .apply(zip_filler)
                                                                     )
prof.head(2)

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code
0,1679576722,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,3500 CENTRAL AVE,NaN,KEARNEY,NE,68847
1,1588667638,1.0,NaN,PILCHER,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,00324


In [15]:
npi_cols = prof.columns.tolist() + taxes.columns[1:].tolist()

In [16]:
npi_cols


['NPI',
 'Entity Type Code',
 'Provider Organization Name (Legal Business Name)',
 'Provider Last Name (Legal Name)',
 'Provider First Name',
 'Provider Middle Name',
 'Provider Name Prefix Text',
 'Provider Name Suffix Text',
 'Provider Credential Text',
 'Provider First Line Business Practice Location Address',
 'Provider Second Line Business Practice Location Address',
 'Provider Business Practice Location Address City Name',
 'Provider Business Practice Location Address State Name',
 'Provider Business Practice Location Address Postal Code',
 'Healthcare Provider Taxonomy Code_1',
 'Healthcare Provider Primary Taxonomy Switch_1',
 'Healthcare Provider Taxonomy Code_2',
 'Healthcare Provider Primary Taxonomy Switch_2',
 'Healthcare Provider Taxonomy Code_3',
 'Healthcare Provider Primary Taxonomy Switch_3',
 'Healthcare Provider Taxonomy Code_4',
 'Healthcare Provider Primary Taxonomy Switch_4',
 'Healthcare Provider Taxonomy Code_5',
 'Healthcare Provider Primary Taxonomy Switch_5'

In [17]:
profile_chunk = pd.merge(prof, primary_taxes)
profile_chunk.columns = [x.lower().replace(' ', '_') for x in profile_chunk.columns]
profile_chunk.head(2)

,npi,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,provider_business_practice_location_address_city_name,provider_business_practice_location_address_state_name,provider_business_practice_location_address_postal_code,taxonomy_code,taxonomy_switch
0,1679576722,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,3500 CENTRAL AVE,NaN,KEARNEY,NE,68847,207X00000X,Y
1,1588667638,1.0,NaN,PILCHER,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,00324,207RC0000X,Y


# hopteam

In [18]:
ht = pd.read_csv('../data/DocGraph_Hop_Teaming_2018.csv', chunksize = 1000)
ht_chunk = next(ht)
ht_chunk.head(1)

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508062167,1730166109,350,370,53.922,72.612


In [19]:
ht_chunk = ht_chunk.reset_index().rename(columns = {'index': 'referral_id'})
from_npi = ht_chunk[['referral_id', 'from_npi']]
to_npi = ht_chunk[['referral_id', 'to_npi']]
referrals = ht_chunk[['referral_id', 
                      'patient_count',
                      'transaction_count',
                      'average_day_wait',
                      'std_day_wait']]
referrals.head(1)

,referral_id,patient_count,transaction_count,average_day_wait,std_day_wait
0,0,350,370,53.922,72.612


# cbsa

In [20]:
zip_cbsa = pd.read_excel('../data/ZIP_CBSA_122021.xlsx')
zip_cbsa.head(1)

,zip,cbsa,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,683,41900,SAN GERMAN,PR,0.999842,1.0,1.0,0.999855


In [21]:
zip_cbsa['zip'] = zip_cbsa['zip'].astype(str).str.zfill(5)
zip_cbsa['cbsa'] = zip_cbsa['cbsa'].astype(str).str.zfill(5)
zip_cbsa = zip_cbsa.rename(columns = {'usps_zip_pref_city': 'city',
                                      'usps_zip_pref_state': 'state'})

# Create SQL database
- profile
- referral_from
- referral_to
- referral
- taxonomy
- zip_cbsa

In [22]:
prof_cols = ['NPI',
             'Entity Type Code',
             'Provider Organization Name (Legal Business Name)',
             'Provider Last Name (Legal Name)',
             'Provider First Name',
             'Provider Middle Name',
             'Provider Name Prefix Text',
             'Provider Name Suffix Text',
             'Provider Credential Text',
             'Provider First Line Business Practice Location Address',
             'Provider Second Line Business Practice Location Address',
             'Provider Business Practice Location Address City Name',
             'Provider Business Practice Location Address State Name',
             'Provider Business Practice Location Address Postal Code']

In [23]:
db = sqlite3.connect('../data/hopteam.sqlite')

At this point, I've melted my data twice making it less wide but much longer. I have a column with a bunch of values that say something like 'Healthcare Provider Taxonomy Code_5' or 'Healthcare Provider Primary Taxonomy Switch_11'. I don't really need all the words, though, since my column labels are already telling me that these are codes or switches; I just need the numbers at the end. I tested a few ways of just getting the numbers, and the most efficient in terms of my computer getting the job done quickly was just grabbing whatever text comes after all the words. So the len('Healthcare Provider Primary Taxonomy Switch_') could just as well have been the number 44, representing the number of characters there. so that could say x[44:] to say "start after all those letters and get me everything to the end"--that is, get me the numbers. The apply( lambda x) basically makes that a function that does the same thing all the way down the dataset.

In [24]:
# Chunk loop through npidata to build the nppes table in the hopteam database
for chunk in tqdm(pd.read_csv('../data/npidata_pfile_20050523-20220213.csv', 
                              usecols = npi_cols,
                              dtype = {'Provider Business Practice Location Address Postal Code': str},
                              chunksize = 100000)):
  
    # Build the primary taxonomy table for this chunk
    taxes = chunk.filter(regex = '^NPI$|(Taxonomy (Code|Switch))')

    taxes_code_long = taxes.melt(id_vars = [col for col in taxes.columns if 'NPI' in col or 'Taxonomy Switch' in col],
               var_name = 'Taxonomy Code Number',
               value_name = 'Taxonomy Code'
              )
    taxes_long = taxes_code_long.melt(id_vars = [col for col in taxes_code_long.columns if 'NPI' in col or 'Taxonomy Code' in col],
                     var_name = 'Taxonomy Switch Number',
                     value_name = 'Taxonomy Switch'
                )

    taxes_long['Taxonomy Code Number'] = taxes_long['Taxonomy Code Number'].apply(lambda x: x[len('Healthcare Provider Taxonomy Code_'):])
    taxes_long['Taxonomy Switch Number'] = taxes_long['Taxonomy Switch Number'].apply(lambda x: x[len('Healthcare Provider Primary Taxonomy Switch_'):])   

    primary_taxes = (taxes_long[(taxes_long['Taxonomy Code Number'] == taxes_long['Taxonomy Switch Number']) 
                    & 
                    (taxes_long['Taxonomy Switch'] == 'Y')]
                    .drop(columns = ['Taxonomy Code Number','Taxonomy Switch Number'])
                )
    
    # Build the prof table for this chunk
    
    prof = chunk.filter(items = prof_cols)
    prof['Provider Business Practice Location Address Postal Code'] = (prof['Provider Business Practice Location Address Postal Code']
                                                                      .apply(zip_filler)
                                                                     )
    
    # Merge prof and taxonomy tables to get profile_chunk
    
    profile_chunk = pd.merge(prof, primary_taxes)
    
    profile_chunk.columns = [x.lower().replace(' ', '_') for x in profile_chunk.columns]
    
    # Append chunk to nppes table in hopteam database
    
    profile_chunk.to_sql('profile', db, if_exists = 'append', index = False)

0it [00:00, ?it/s]

/Users/ntaherkh/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (79,82,83,86,87,90,91,94,95,98,99,102,103,106) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/ntaherkh/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (71,74,75,78,79,82,83,86,87,90,91,94,95,98,99,102,103,106) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/ntaherkh/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (91,94,95,98,99,102,103,106) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/ntaherkh/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (75,78,79,82,8

In [25]:
for chunk in tqdm(pd.read_csv('../data/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)):
    ht_chunk = chunk.reset_index().rename(columns = {'index': 'referral_id'})
    from_npi_chunk = ht_chunk[['referral_id', 'from_npi']]
    to_npi_chunk = ht_chunk[['referral_id', 'to_npi']]
    referrals_chunk = ht_chunk[['referral_id', 
                          'patient_count',
                          'transaction_count',
                          'average_day_wait',
                          'std_day_wait']]
    
    from_npi_chunk.to_sql('referral_from', db, if_exists = 'append', index = False)
    to_npi_chunk.to_sql('referral_to', db, if_exists = 'append', index = False)
    referrals_chunk.to_sql('referrals', db, if_exists = 'append', index = False)

0it [00:00, ?it/s]

In [26]:
tax_detail.to_sql('taxonomy', db, if_exists = 'append', index = False)
zip_cbsa.to_sql('zip_cbsa', db, if_exists = 'append', index = False)

/Users/ntaherkh/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [27]:
db.close()